In [ ]:
# %pip install scikit-surprise==1.1.0 # pickle

In [ ]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

In [ ]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

In [ ]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

In [ ]:
df.rating.max(), df.rating.min()

In [ ]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

In [ ]:
trainset = data.build_full_trainset()
algo.fit(trainset)

In [ ]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

In [ ]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

In [ ]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

In [ ]:
import pickle

In [ ]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [ ]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [ ]:
model_algo.predict(uid=userId, iid=movieId).est

In [ ]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)